# Isaac Sim Example
In this notebook, we simulate the robot with joint velocity interface using the Nvidia Isaac Sim. Before running this notebook, make sure you follow through the installation procedure and then run the following command to start the simulation node:

```bash
./fr3py.sh --sim
```

This will launch the Isaac Sim environment. After the environment is fully launched, use the `FR3IsaacSim` interface class to interact with it. The API is fully compatible with the `FR3Real` class.

In [1]:
from FR3Py.sim.interface import FR3IsaacSim
robot = FR3IsaacSim()

ef_camera_distance_to_camera
ef_camera_rgb
perspective_rgb


Note that in addition to the possibility of reading the states, we can also read image data from the cameras placed inside the simulator. Note that the configuration and placement of these cameras alongside the scene configuration, robot stand, and etc may be modified through editing the 'FR3Py/sim/isaac/sim_config.yaml' configuration file.

Let's first just send random velocity commands to the robot and see it's movements:

In [9]:
import cv2
import numpy as np
for i in range(1000):
    state = robot.getStates()
    robot.sendCommands(np.random.randn(9))
    imgs =  robot.readCameras()
    for stream_name, img in imgs.items():
        cv2.imshow(f'{stream_name}', img)
    cv2.waitKey(1)

cv2.destroyAllWindows()

Now that we can command the robot to have random motion, let's control the end-effector pose using the same controller we used for controlling the real robot:

In [31]:
import time
import numpy as np
from FR3Py.controllers.jacobianPseudoInv import WaypointController

controller = WaypointController(kp=1.5)

# Read the initial state of the robot At the beginning, of the simulation, the robot may not be in the proper initial state.
# So, we need to send zero command and read the state of the robot enough times to get the proper initial state. 
# TODO: This should be fixed in the future through the addition of a reset function in the interface.
 
for i in range(100):
    time.sleep(0.01)
    robot.sendCommands(np.zeros(9))
    state = robot.getStates()

if state is not None:
    print('starting the controller')
    q, dq = state['q'], state['dq']
    p0 = controller.robot.getInfo(q,dq)['P_EE']
    R0 = controller.robot.getInfo(q,dq)['R_EE']
    T0 = np.vstack([np.hstack([R0, p0.reshape(3,1)]), np.array([0,0,0,1])])
    start_time = time.time()
    while time.time()-start_time < 20:
        time.sleep(0.01)
        t = time.time() - start_time
        x = 0.1 * np.sin(t)
        T= T0 @ np.vstack([np.hstack([np.eye(3), np.array([0,0,x]).reshape(3,1)]), np.array([0,0,0,1])])
        state = robot.getStates()
        q, dq = state['q'], state['dq']
        cmd = controller.compute(q, dq, T_cmd=T)
        robot.sendCommands(cmd)

    robot.sendCommands(np.zeros(9))
    print('Demo ended.')
else:
    print('State is None. check the connection')

starting the controller
Demo ended.
